In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
log.setLevel(logging.DEBUG)

In [ ]:
def plot_gwb(fobs, gwb, ax=None):
    if ax is None:
        fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')
    else:
        fig = ax.figure

    xx = (fobs[1:] + fobs[:-1]) * 0.5
    ax.plot(xx, np.median(gwb, axis=1))

    ax.fill_between(xx, *np.percentile(gwb, [25, 75], axis=1), alpha=0.2)

    return fig, ax

# Basics

In [ ]:
NUM = 1e2
TIME = 1.0 * GYR
RMAX = 3.0e3 * PC
mtot = [6, 10]
mrat = [-4, 0]
redz = [-2, 1]

NUM = int(NUM)
units = [MSOL, 1.0, 1.0]
mtot, mrat, redz = [
    10.0**np.random.uniform(*xx, NUM) * un
    for xx, un in zip([mtot, mrat, redz], units)
]
sepa = RMAX * np.ones_like(mtot)

fixed = holo.evolution.Fixed_Time(TIME, mtot, mrat, redz, sepa)
norm = fixed._norm
print(f"{utils.stats(norm)=}")

result_time = fixed._time_total(
    norm, mtot, mrat,
    fixed._rchar, fixed._gamma_sc, fixed._gamma_df, sepa
)
print(f"{utils.stats(result_time/GYR)=}")
error = (TIME - result_time) / TIME
print(f"{utils.stats(error)=}")

fig, axes = plt.subplots(figsize=[12, 5], ncols=2)

ax = axes[0]
ax.set(title='Hardening Normalization', xlabel='$\log_{10}(A)$')
kale.plot.dist1d(np.log10(norm), ax=ax, density=False)

ax = axes[1]
ax.set(title='Time Error', xlabel='$\Delta t / t$')
kale.plot.dist1d(error, ax=ax, density=False)

plt.show()

In [ ]:
select = 30
rads = np.logspace(*np.log10([fixed._TIME_TOTAL_RMIN, RMAX]), 200)

dadt = fixed.dadt(mtot[np.newaxis, :], mrat[np.newaxis, :], rads[:, np.newaxis])
tau = rads[:, np.newaxis] / -dadt

tau = tau / GYR
xx = rads / PC

select = np.min([mtot.size, select])
fig, ax = plot.figax(ylim=[1e-6, 1e1])
ax.plot(xx, tau.T[np.random.choice(mtot.size, select, replace=False)].T, 'k-', alpha=0.25, lw=0.5)
ax.plot(xx, np.median(tau, axis=1))
ax.fill_between(xx, *np.percentile(tau, [25, 75], axis=1), alpha=0.2)

plt.show()

## Test Interpolants

In [ ]:
norm_true = fixed._norm
interp, backup = fixed._calculate_norm_interpolant(fixed._rchar, fixed._gamma_sc, fixed._gamma_df)

In [ ]:
vals = [np.log10(mtot/MSOL), np.log10(mrat), TIME/GYR, np.log10(sepa/PC)]
vals = np.broadcast_arrays(*vals)
vals = np.array(vals)

In [ ]:
print(f"{utils.stats(norm_true)=}")
norm_interp = 10.0 ** interp(vals.T)
norm_backup = 10.0 ** backup(vals.T)
print(f"{utils.stats(norm_interp)=}")
print(f"{utils.stats(norm_backup)=}")
error_interp = (norm_interp - norm_true) / norm_true
error_backup = (norm_backup - norm_true) / norm_true
print(f"{utils.stats(error_interp)=}")
print(f"{utils.stats(error_backup)=}")

# SAM

In [ ]:
fobs = utils.nyquist_freqs(10.0*YR, 0.2*YR)

In [ ]:
gsmf = holo.sam.GSMF_Schechter()               # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()                 # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()                 # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()    # M-MBulge Relation            (MMB)

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)  # , shape=10)

In [ ]:
fc = utils.nyquist_freqs(10, 0.2)
print(fc)
df = fc[0]/2
fe = fc - df
fe = np.concatenate([fe, [fc[-1] + df]])
print(fe)
print(fc.size, fe.size)

In [ ]:
TIME = 1e-2 * GYR
sepa_init = 1e4 * PC

sam_small = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)
fixed_small = holo.evolution.Fixed_Time.from_sam(sam_small, TIME, sepa_init=sepa_init)
gwb_small = sam_small.gwb(fobs, realize=3, hard=fixed_small)    # calculate many different realizations
fig, ax = plot_gwb(fobs, gwb_small)

In [ ]:
mt, mr, rz = [gg.ravel() for gg in sam_small.grid]
# norm = fixed._get_norm_chunk(
#     TIME, mt, mr, fixed._rchar, fixed._gamma_sc, fixed._gamma_df, sepa_init
# )
tot_time = fixed_small._time_total(
    fixed_small._norm, mt, mr,
    fixed_small._rchar, fixed_small._gamma_sc, fixed_small._gamma_df,
    sepa_init
)
print(f"{TIME/GYR=}")
print(f"{utils.stats(tot_time/GYR)=}")

In [ ]:
fixed = holo.evolution.from_sam(sam, 1e2 * GYR)

In [ ]:
gwb = sam.gwb(fobs, realize=3, hard=fixed)    # calculate many different realizations

In [ ]:
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

xx = (fobs[1:] + fobs[:-1]) * 0.5
ax.plot(xx, np.median(gwb, axis=1))

ax.fill_between(xx, *np.percentile(gwb, [25, 75], axis=1), alpha=0.2)

plt.show()